In [1]:
import win32com.client as win32
import pandas as pd
import os
import calendar
from datetime import datetime
from dateutil.relativedelta import relativedelta
from data.data_mockers import get_data_captiatalization
import pandas as pd

if not os.path.exists(os.path.join(os.getcwd(), "output_path")):
    print("Creating an output path")
    os.makedirs("output_path", exist_ok=True)

In [4]:
from svgwrite import Drawing
from svgwrite.container import Group
from svgwrite.text import Text

# Table data as rows of values
table_data = get_data_captiatalization()

# Define styling
cell_width = 100
cell_height = 40
font_size = 13
title_font_size = 30
num_cols = len(table_data[0])
num_rows = len(table_data)

# Canvas size
width = cell_width * num_cols + 50
height = cell_height * (num_rows + 2)  # +2 for spacing and title

# Create drawing
dwg = Drawing("capitalization_table_final.svg", size=(width, height))
group = Group()

# Draw title
group.add(Text("Current Capitalization", insert=(20, title_font_size + 20), font_size=title_font_size, font_weight="bold"))

# Colors
header_color = "#009CA6"
even_row_color = "#FFFFFF"
highlight_color = "#FDE0C2"
footer_color = "#E4C4DC"
text_color = "#000"

# Draw table
start_y = title_font_size + 50
for row_idx, row in enumerate(table_data):
    y = start_y + row_idx * cell_height
    for col_idx, cell in enumerate(row):
        x = 20 + col_idx * cell_width

        # Determine background color
        if row_idx == 0:
            fill = header_color
            font_weight = "bold"
            font_fill = "white"
        elif row_idx in [6, 7, 11, 12]:
            fill = highlight_color
            font_weight = "bold"
            font_fill = text_color
        elif row_idx == 13:
            fill = footer_color
            font_weight = "bold"
            font_fill = text_color
        else:
            fill = even_row_color
            font_weight = "normal"
            font_fill = text_color

        # Draw cell background
        group.add(dwg.rect(insert=(x, y), size=(cell_width, cell_height), fill=fill, stroke="black", stroke_width=1))

        # Draw cell text
        if cell:
            group.add(Text(
                cell,
                insert=(x + 5, y + cell_height / 2 + font_size / 2.5),
                font_size=font_size,
                fill=font_fill,
                font_weight=font_weight
            ))

dwg.add(group)
dwg.save()
"SVG with adjusted bottom padding saved as capitalization_table_final_adjusted.svg"


'SVG with adjusted bottom padding saved as capitalization_table_final_adjusted.svg'

In [25]:
def get_calendar_lookup():
    lookup_df = pd.read_excel("data/events_data.xlsx")
    lookup_df["Date"] = lookup_df["Date"].apply(lambda x: str(datetime.date(x)))
    lookup_dict = {}
    for date, event in lookup_df.values:
        try:
            lookup_dict[event].append(date)
        except Exception as e:
            lookup_dict[event] = []
    return lookup_dict

In [34]:
def check_for_events(sample_date, lookup_dictionary):
    for key in lookup_dictionary.keys():
        if sample_date in lookup_dictionary[key]:
            return key
    return None

In [37]:
import svgwrite
import calendar

def create_calendar_svg(svg_path):
    # Increased width further to ensure no clipping
    canvas_width = 1180  # Fix for right-side border cutoff
    canvas_height = 750
    dwg = svgwrite.Drawing(svg_path, profile='tiny', size=(f"{canvas_width}px", f"{canvas_height}px"))

    months = ["March", "April", "May", "June", "July", "August"]
    year = 2025
    colors = {
        "execution": "#D9D9D9",  # Light grey
        "cpi":       "#FFFFE0",  # Light yellow
        "ppi":       "#ED7D31",  # Dark orange
        "hld":       "#C6E0B4"   # Light green     # Light green
    }

    cell_width = 53
    cell_height = 40
    x_margin = 25
    y_margin = 50
    top_extra_margin = 30  # For visibility of month titles
    stroke_width = 0.8
    lookup_calendar = get_calendar_lookup()

    
    for idx, month_name in enumerate(months):
        month = idx + 3
        cal = calendar.Calendar(firstweekday=0)
        month_days = list(cal.itermonthdays(year, month))

        grid_col = idx % 3
        grid_row = idx // 3

        x_offset = grid_col * (cell_width * 7 + x_margin)
        y_offset = top_extra_margin + grid_row * (cell_height * 7 + y_margin + 35)

        # Month title
        dwg.add(dwg.text(f"{month_name} {year}",
                         insert=(x_offset, y_offset - 15),
                         font_size="13px", font_weight="bold"))

        # Weekday headers
        weekdays = ["M", "T", "W", "Th", "F", "Sa", "Su"]
        for i, day in enumerate(weekdays):
            dwg.add(dwg.text(day,
                             insert=(x_offset + i * cell_width + 3, y_offset + 10),
                             font_size="9px", font_weight="bold"))

        # Calendar day boxes
        col = 0
        row = 1
        key = None
        for day in month_days:
            if day == 0:
                col += 1
                if col >= 7:
                    col = 0
                    row += 1
                continue

            rect_x = x_offset + col * cell_width
            rect_y = y_offset + row * cell_height

            dwg.add(dwg.rect(insert=(rect_x, rect_y),
                             size=(cell_width, cell_height),
                             fill="#e0f7fa", stroke="gray", stroke_width=stroke_width))

            date_field = f"{day}-{month_name}-{year}"
            date_obj = datetime.strptime(date_field, '%d-%B-%Y')
            formatted_date = date_obj.strftime('%Y-%m-%d')
            key = check_for_events(sample_date=formatted_date, lookup_dictionary=get_calendar_lookup())
            if key:
                dwg.add(dwg.text(str(day) + "\n" + key,
                                 insert=(rect_x + 3, rect_y + 13),
                                 font_size="8px"))
            else:
                dwg.add(dwg.text(str(day),
                                 insert=(rect_x + 3, rect_y + 13),
                                 font_size="8px"))

            col += 1
            if col >= 7:
                col = 0
                row += 1

    dwg.save()
    print(f"✅ Calendar saved to: {svg_path}")


# Generate SVG
create_calendar_svg("calendar_march_to_august_2025.svg")

✅ Calendar saved to: calendar_march_to_august_2025.svg


In [29]:
date_field

NameError: name 'date_field' is not defined

In [2]:
def create_ppt_with_template(template_path):#, output_path, df):
    if not os.path.exists(template_path):
        raise FileNotFoundError(f"❌ Template not found at {template_path}")

    # Launch PowerPoint
    ppt_app = win32.Dispatch("PowerPoint.Application")
    ppt_app.Visible = True

    # Open the template as the base presentation
    presentation_sample = ppt_app.Presentations.Open(template_path)
    return presentation_sample

In [5]:
def create_current_captiatlization(presentation_sample):

    output_folder = os.path.join(os.getcwd(), "output_path")
    slide = presentation_sample.Slides.Add(presentation_sample.Slides.Count + 1, 1)
    title_shape = slide.Shapes.Title
    title_shape.TextFrame.TextRange.Text = "Current Capitalization"
    title_shape.Left = 10    # distance from the left (in points)
    title_shape.Top = 30     # distance from the top (in points)
    title_shape.Width = 400  # optional: adjust width
    title_shape.Height = 30  # optional: adjust height
    
    highlight_rows = {"First Lien Debt", "Net First Lien Debt", "Total Debt", "Total Net Debt"}
    footer_row = "LTM Run-Rate Adj. EBITDA"
    data = get_data_captiatalization()
    
    ## Getting number of rows and cols 
    rows = len(data)
    cols = len(data[0])
    
    left = 20     # points
    top = 70
    width = 900
    height = 300
    table_shape = slide.Shapes.AddTable(rows, cols, left, top, width, height)
    table = table_shape.Table

    # Filling Data here
    for i, row_data in enumerate(data):
        for j, cell_text in enumerate(row_data):
            cell = table.Cell(i + 1, j + 1)
            cell.Shape.TextFrame.TextRange.Text = cell_text
            cell.Shape.TextFrame.TextRange.Font.Size = 8
            cell.Shape.TextFrame.TextRange.Font.Bold = True if i == 0 or row_data[0] in highlight_rows or row_data[0] == footer_row else False
            cell.Shape.TextFrame.VerticalAnchor = 3  # center vertically
            cell.Shape.TextFrame.TextRange.ParagraphFormat.Alignment = 2 if j != 0 else 1  # right or left align
    
        # Row shading
        if row_data[0] in highlight_rows:
            for j in range(cols):
                table.Cell(i + 1, j + 1).Shape.Fill.ForeColor.RGB = 0xCCE5FF  # light blue
        elif row_data[0] == footer_row:
            for j in range(cols):
                table.Cell(i + 1, j + 1).Shape.Fill.ForeColor.RGB = 0xD9CCE3  # light purple

    presentation.SaveAs(os.path.join(output_folder, "captiatlization_sheet.pptx"))
    return presentation

In [6]:
presentation = create_ppt_with_template(template_path = r"D:\\ppt_automate\\template_path\\SampleTemplate.pptx")
presentation = create_current_captiatlization(presentation)

In [7]:
def get_calendar_lookup():
    lookup_df = pd.read_excel("data/events_data.xlsx")
    lookup_df["Date"] = lookup_df["Date"].apply(lambda x: str(datetime.date(x)))
    lookup_dict = {}
    for date, event in lookup_df.values:
        try:
            lookup_dict[event].append(date)
        except Exception as e:
            lookup_dict[event] = []
    return lookup_dict

In [8]:
def check_for_events(sample_date, lookup_dictionary):
    for key in lookup_dictionary.keys():
        if sample_date in lookup_dictionary[key]:
            return key
    return None

In [14]:
# Start date
def get_calendar_slide(presentation_sample, date_str = '2025-04-01'):

    output_folder = os.path.join(os.getcwd(), "output_path")
    start_date = datetime.strptime(date_str, '%Y-%m-%d')
    
    months = []
    for i in range(6):
        date = start_date + relativedelta(months=i)
        months.append({
            'month_name': date.strftime('%B'),     # e.g. 'April'
            'month_number': date.strftime('%m'),   # e.g. '04'
            'year': date.strftime('%Y')            # e.g. '2025'
        })
    
    colors = {
        "execution": (217, 217, 217),  # Light grey
        "cpi": (255, 255, 224),         # Light yellow
        "ppi": (237, 125, 49),         # Dark orange
        "hld": (198, 224, 180)     # Light green
    }
    lookup_calendar = get_calendar_lookup()
    # Initialize PowerPoint
    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    slide = presentation_sample.Slides.Add(presentation_sample.Slides.Count + 1, 12)
    
    # Title
    title_box = slide.Shapes.AddTextbox(1, 20, 10, 700, 50)
    title_box.TextFrame.TextRange.Text = "*EXAMPLE* - Overview of Execution Windows"
    title_box.TextFrame.TextRange.Font.Size = 24
    
    # Grid layout parameters
    cal_per_row = 3
    calendar_width = 200
    calendar_height = 160
    cell_spacing = 1
    cell_w = (calendar_width - (6 * cell_spacing)) / 7  # 7 columns
    cell_h = (calendar_height - 25 - (5 * cell_spacing)) / 6  # 6 rows
    
    margin_left = 50
    margin_top = 70
    h_spacing = 60
    v_spacing = 50  # 🔻 Reduced from 70 to make rows closer
    
    # RGB converter
    def rgb(r, g, b):
        return r + (g << 8) + (b << 16)
    
    # Create calendars
    for i, month in enumerate(months):
        month_idx = list(calendar.month_name).index(month['month_name'])
        month_matrix = calendar.monthcalendar(int(month['year']), month_idx)
    
        row = i // cal_per_row
        col = i % cal_per_row
    
        left = margin_left + col * (calendar_width + h_spacing)
        top = margin_top + row * (calendar_height + v_spacing)
    
        # Month Title
        title = slide.Shapes.AddTextbox(1, left, top, calendar_width, 20)
        title.TextFrame.TextRange.Text = f"{month['month_name']} {month['year']}"
        title.TextFrame.TextRange.Font.Size = 14
        title.TextFrame.TextRange.Font.Bold = True
    
        # Day cells
        for week_idx, week in enumerate(month_matrix):
            for day_idx, day in enumerate(week):
                if day == 0:
                    continue
                    
                key = None
                shape = slide.Shapes.AddTextbox(
                    1,
                    left + day_idx * (cell_w + cell_spacing),
                    top + 25 + week_idx * (cell_h + cell_spacing),
                    cell_w,
                    cell_h
                )
                
                date_looped = str(datetime.date(datetime.strptime(month['year'] + '-' + month['month_number'] + '-' + str(day), '%Y-%m-%d')))
                key = check_for_events(sample_date=date_looped, lookup_dictionary=get_calendar_lookup())
                if key:
                    shape.TextFrame.TextRange.Text = str(day) + f"\n{key}"
                    shape.Fill.ForeColor.RGB = rgb(*colors[key.lower()])
                    shape.TextFrame.TextRange.Font.Bold = True
                else:
                    shape.TextFrame.TextRange.Text = str(day) + f"\n "
                    shape.Fill.ForeColor.RGB = rgb(*colors["execution"])
                    
                shape.TextFrame.TextRange.Font.Size = 6
    
                # Bottom-right align
                shape.TextFrame.TextRange.ParagraphFormat.Alignment = 3  # Right
                shape.TextFrame.VerticalAnchor = 3  # Bottom
    
                # Fill with Execution Window color
                shape.Line.Visible = False
    
    # Add legend boxes
    legend_items = [
        ("CPI", colors["cpi"]),
        ("PPI", colors["ppi"]),
        ("Holiday", colors["hld"]),
    ]
    
    legend_top = margin_top + 2 * (calendar_height + v_spacing) + 10
    legend_left = margin_left
    
    for i, (label, color_rgb) in enumerate(legend_items):
        box = slide.Shapes.AddShape(1, legend_left + i * 180, legend_top, 20, 20)
        box.Fill.ForeColor.RGB = rgb(*color_rgb)
        box.Line.Visible = False
    
        label_box = slide.Shapes.AddTextbox(1, legend_left + i * 180 + 25, legend_top, 150, 20)
        label_box.TextFrame.TextRange.Text = label
        label_box.TextFrame.TextRange.Font.Size = 12

    
    presentation.SaveAs(os.path.join(output_folder, "calendar_sheet.pptx"))
    print("✅ Calendar Slides Done.")

In [15]:
presentation = create_ppt_with_template(template_path = r"D:\\ppt_automate\\template_path\\MorganStanley-SampleTemplate.pptx")
presentation = get_calendar_slide(presentation, date_str = '2025-05-01')

✅ Calendar Slides Done.


In [4]:
import win32com.client as win32
import os

# Data for team members
team = [
    {
        "name": "Mr A",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mra@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    {
        "name": "Mr B",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mrB@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    {
        "name": "Mr A",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mra@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    {
        "name": "Mr B",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mrB@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
        {
        "name": "Mr A",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mra@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    {
        "name": "Mr B",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mrB@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
        {
        "name": "Mr A",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mra@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    {
        "name": "Mr B",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mrB@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
        {
        "name": "Mr A",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mra@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    {
        "name": "Mr B",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mrB@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
        {
        "name": "Mr A",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mra@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    {
        "name": "Mr B",
        "title": "Executive Chairman,\nLeela Palace",
        "phone": "+1 000 000-0000",
        "email": "mrB@abc.com",
        "image": "https://upload.wikimedia.org/wikipedia/commons/7/7c/Profile_avatar_placeholder_large.png"
    },
    # Add other members similarly...
]

# Initialize PowerPoint
ppt = win32.Dispatch("PowerPoint.Application")
ppt.Visible = True
presentation = ppt.Presentations.Add()

# Add slide
slide = presentation.Slides.Add(1, win32.constants.ppLayoutBlank)

# Constants for positioning
left_margin = 50
top_margin = 50
image_size = 100
x_spacing = 180
y_spacing = 220

# Download image to temp
import urllib.request

def download_image(url, filename):
    urllib.request.urlretrieve(url, filename)
    return filename

# Add title
slide.Shapes.AddTextbox(1, left_margin, 10, 800, 40).TextFrame.TextRange.Text = "The Leela Palcace"

# Loop through team members and insert them
for i, member in enumerate(team):
    col = i % 5
    row = i // 5
    left = left_margin + col * x_spacing
    top = top_margin + row * y_spacing

    # Download image temporarily
    img_path = os.path.join(os.getcwd(), f"temp_img_{i}.jpg")
    download_image(member["image"], img_path)

    # Add image and crop to circle
    shape = slide.Shapes.AddPicture(img_path, False, True, left, top, image_size, image_size)
    picture_format = shape.PictureFormat
    shape.LockAspectRatio = True
    shape.Width = image_size
    shape.Height = image_size
    shape.PictureFormat.Crop.PictureHeight = image_size
    shape.PictureFormat.Crop.PictureWidth = image_size

    # Use crop to simulate circle by overlaying a circle shape mask
    circle = slide.Shapes.AddShape(9, left, top, image_size, image_size)  # msoShapeOval = 9
    circle.Fill.UserPicture(img_path)
    circle.Line.Visible = False
    shape.Delete()  # Remove the original

    # Add name
    slide.Shapes.AddTextbox(1, left, top + image_size + 5, 150, 20).TextFrame.TextRange.Text = member["name"]
    
    # Add title
    slide.Shapes.AddTextbox(1, left, top + image_size + 25, 150, 40).TextFrame.TextRange.Text = member["title"]

    # Add phone/email
    slide.Shapes.AddTextbox(1, left, top + image_size + 65, 150, 40).TextFrame.TextRange.Text = f"{member['phone']}\n{member['email']}"

# Save presentation
output_path = os.path.join(os.getcwd(), "SampleTeam_Team.pptx")
presentation.SaveAs(output_path)
print(f"✅ Presentation saved to {output_path}")

✅ Presentation saved to D:\ppt_automate\SampleTeam_Team.pptx


In [32]:
slide.Shapes.AddTextbox()

com_error: (-2147352561, 'Parameter not optional.', None, None)

In [42]:
month_height / 6

35.0

In [ ]:
def create_calendar(presentation_sample):
    # Add main title
    slide = presentation.Slides.Add(presentation.Slides.Count + 1, 12)
    title = slide.Shapes.AddTextbox(1, 20, 10, 900, 40)
    title.TextFrame.TextRange.Text = "*EXAMPLE* - IPO Timeline\n"
    title.TextFrame.TextRange.Font.Size = 24
    title.TextFrame.TextRange.Font.Bold = True

    ## Add sub Title
    title = slide.Shapes.AddTextbox(1, 20, 40, 900, 40)
    title.TextFrame.TextRange.Text = " Q2'26 IPO"
    title.TextFrame.TextRange.Font.Size = 24
    title.TextFrame.TextRange.Font.Bold = True
    title.TextFrame.TextRange.Font.Color.RGB = 0xE16941

    return presentation_sample

In [10]:
# Add main title
slide = presentation.Slides.Add(presentation.Slides.Count + 1, 12)
title = slide.Shapes.AddTextbox(1, 20, 10, 900, 40)
title.TextFrame.TextRange.Text = "*EXAMPLE* - IPO Timeline\n"
title.TextFrame.TextRange.Font.Size = 24
title.TextFrame.TextRange.Font.Bold = True

In [12]:
title = slide.Shapes.AddTextbox(1, 20, 40, 900, 40)
title.TextFrame.TextRange.Text = " Q2'26 IPO"
title.TextFrame.TextRange.Font.Size = 24
title.TextFrame.TextRange.Font.Bold = True
title.TextFrame.TextRange.Font.Color.RGB = 0xE16941

In [14]:
start_date, end_date = '2025-06-01', '2026-05-01'
months = [
    (2025, 6), (2025, 7), (2025, 8), (2025, 9),
    (2025, 10), (2025, 11), (2025, 12), (2026, 1),
    (2026, 2), (2026, 3), (2026, 4), (2026, 5)
]

In [54]:
slide = presentation.Slides.Add(presentation.Slides.Count + 1, 1)
title_shape = slide.Shapes.Title
title_shape.TextFrame.TextRange.Text = "Current Capitalization"
title_shape.Left = 10    # distance from the left (in points)
title_shape.Top = 30     # distance from the top (in points)
title_shape.Width = 400  # optional: adjust width
title_shape.Height = 30  # optional: adjust height

In [55]:
highlight_rows = {"First Lien Debt", "Net First Lien Debt", "Total Debt", "Total Net Debt"}
footer_row = "LTM Run-Rate Adj. EBITDA"
data = get_data_captiatalization()

## Getting number of rows and cols 
rows = len(data)
cols = len(data[0])

left = 20     # points
top = 70
width = 900
height = 300
table_shape = slide.Shapes.AddTable(rows, cols, left, top, width, height)
table = table_shape.Table

In [56]:
for i, row_data in enumerate(data):
    for j, cell_text in enumerate(row_data):
        cell = table.Cell(i + 1, j + 1)
        cell.Shape.TextFrame.TextRange.Text = cell_text
        cell.Shape.TextFrame.TextRange.Font.Size = 8
        cell.Shape.TextFrame.TextRange.Font.Bold = True if i == 0 or row_data[0] in highlight_rows or row_data[0] == footer_row else False
        cell.Shape.TextFrame.VerticalAnchor = 3  # center vertically
        cell.Shape.TextFrame.TextRange.ParagraphFormat.Alignment = 2 if j != 0 else 1  # right or left align

    # Row shading
    if row_data[0] in highlight_rows:
        for j in range(cols):
            table.Cell(i + 1, j + 1).Shape.Fill.ForeColor.RGB = 0xCCE5FF  # light blue
    elif row_data[0] == footer_row:
        for j in range(cols):
            table.Cell(i + 1, j + 1).Shape.Fill.ForeColor.RGB = 0xD9CCE3  # light purple

In [35]:
rows = len(data)
cols = len(data[0])

In [37]:
rows, cols

(14, 13)

In [38]:
left = 50     # points
top = 100
width = 900
height = 300
table_shape = slide.Shapes.AddTable(rows, cols, left, top, width, height)
table = table_shape.Table

In [ ]:
import win32com.client as win32
import pythoncom

# Start PowerPoint
pythoncom.CoInitialize()
ppt_app = win32.Dispatch("PowerPoint.Application")
ppt_app.Visible = True
presentation = ppt_app.Presentations.Add()

# Add slide
slide = presentation.Slides.Add(1, 1)  # 1 = ppLayoutText
slide.Shapes.Title.TextFrame.TextRange.Text = "Current Capitalization"

# Table data (13 rows + header, 13 columns)
data = [
    ["$MM", "Amount", "xEBITDA", "Maturity", "Rating", "Floor", "Pricing", "Call Price", "Next Call Date", "Next Call Price", "Price", "Yield", "YTW Date"],
    ["Cash and Cash Equivalents", "118", "", "", "", "", "", "", "", "", "", "", ""],
    ["$750MM Revolver", "-", "", "Jun-28", "B1 / B", "0.00%", "S + 300", "", "", "100.000", "100.250", "6.74%", "-"],
    ["C$140MM Revolver", "-", "", "Jun-28", "B1 / B", "0.00%", "CDOR + 300", "", "", "", "", "", ""],
    ["Term Loan B", "5,572", "", "Jun-30", "B1 / B", "0.75%", "S + 250", "101.000", "Jul-25", "100.000", "100.250", "6.74%", "-"],
    ["Senior Secured Notes", "3,275", "", "Jun-30", "B1 / B", "-", "7.250%", "", "Jun-26", "103.625", "103.000", "6.24%", "Jun-28"],
    ["First Lien Debt", "8,847", "5.1x", "", "", "", "", "", "", "", "", "", ""],
    ["Net First Lien Debt", "8,729", "5.1x", "", "", "", "", "", "", "", "", "", ""],
    ["Senior Unsecured Notes", "550", "", "Dec-29", "Caa1 / B-", "-", "5.625%", "102.813", "Dec-25", "101.406", "97.500", "6.23%", "Dec-29"],
    ["Senior Unsecured Notes", "1,900", "", "Jan-32", "Caa1 / B-", "-", "7.375%", "", "Jan-27", "103.688", "102.750", "6.58%", "Jan-29"],
    ["Other Debt", "3", "", "", "", "", "", "", "", "", "", "", ""],
    ["Total Debt", "11,300", "6.6x", "", "", "", "", "", "", "", "", "", ""],
    ["Total Net Debt", "11,182", "6.5x", "", "", "", "", "", "", "", "", "", ""],
    ["LTM Run-Rate Adj. EBITDA", "1,721", "", "", "", "", "", "", "", "", "", "", ""]
]

highlight_rows = {"First Lien Debt", "Net First Lien Debt", "Total Debt", "Total Net Debt"}
footer_row = "LTM Run-Rate Adj. EBITDA"

rows = len(data)
cols = len(data[0])

# Add table
left = 50     # points
top = 100
width = 900
height = 300
table_shape = slide.Shapes.AddTable(rows, cols, left, top, width, height)
table = table_shape.Table

# Fill table
for i, row_data in enumerate(data):
    for j, cell_text in enumerate(row_data):
        cell = table.Cell(i + 1, j + 1)
        cell.Shape.TextFrame.TextRange.Text = cell_text
        cell.Shape.TextFrame.TextRange.Font.Size = 10
        cell.Shape.TextFrame.TextRange.Font.Bold = True if i == 0 or row_data[0] in highlight_rows or row_data[0] == footer_row else False
        cell.Shape.TextFrame.VerticalAnchor = 3  # center vertically
        cell.Shape.TextFrame.TextRange.ParagraphFormat.Alignment = 2 if j != 0 else 1  # right or left align

    # Row shading
    if row_data[0] in highlight_rows:
        for j in range(cols):
            table.Cell(i + 1, j + 1).Shape.Fill.ForeColor.RGB = 0xCCE5FF  # light blue
    elif row_data[0] == footer_row:
        for j in range(cols):
            table.Cell(i + 1, j + 1).Shape.Fill.ForeColor.RGB = 0xD9CCE3  # light purple

print("✅ Slide generated successfully.")

In [22]:
df = get_data()
create_ppt_with_template(template_path = r"D:\\ppt_automate\\template_path\\SampleTemplate.pptx",
                         output_path="output_path/", df=df)

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)

In [21]:
os.getcwd()

'D:\\ppt_automate'

In [5]:
get_data()

,Month,Sales
0,Jan,120
1,Feb,150
2,Mar,180
3,Apr,130
4,May,200
